# 1. Deep Neural Net

## 1.1 Import libraries

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## 1.2 Load and parse data

In [ ]:
# load dataset, 130 sample X dimension (130, 600), Y dimension (130,1)
df = pd.read_csv('test.csv')
dataset = df.values
# split into input (X) and output (Y) variables, and training/test data
X = dataset[:, 1:601]
Y = dataset[:, 602]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)
print (X_train.shape,Y_train.shape,Y_test.shape,X_test.shape)

## 1.3 Build Deep NN and implementation

In [ ]:
# # create neep NN model
model = Sequential([Dense(32, activation='relu', input_shape = (600, )), 
                   Dense(32,  activation='relu'),
                   Dense(1, activation='sigmoid'),
                   ])
# Compile model
model.compile(optimizer='adam', loss='mean_squared_error',
             metrics = ['accuracy'])

# fitting model and evaluation
hist = model.fit(X_train, Y_train, batch_size = 16, epochs = 500, 
                validation_data =(X_test, Y_test))



## 1.4 Plot loss and accuracy

In [ ]:
#loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

#accuracy
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='lower right')
plt.show()

# 2. PLSR, Ridge Regress, RF, SVM and ELR

## 2.1 Import libraries

In [ ]:
# importing libraries for above regression methos 

from sys import stdout
from scipy.signal import savgol_filter #for derivative calculaiton

import matplotlib.pyplot as plt

import sklearn.cross_decomposition
from sklearn.cross_decomposition import  PLSRegression
from sklearn import linear_model
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR

## 2.2 PLSR
### 2.2.1 Load and parse data

In [ ]:
#original raw CA data (130 samples)

data = pd.read_csv('test.csv')
X = data.values[:, 1:601]
Y = data['Gs']



### 2.2.2 Plotting spectral data

In [ ]:
wl = np.arange(400, 1000, 1)

#plot reflectance spectra

with plt.style.context(('ggplot')):
    plt.plot(wl, X.T)
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('Reflectance')
plt.show()

### 2.2.3 Calculate 1st/2nd derivatives and plotting

In [ ]:
# Calculate first or second derivative (look at parameter values deriv)
X2 = savgol_filter(X, 17, polyorder = 2,deriv=2)
# Plot second derivative
plt.figure(figsize=(8,4.5))
with plt.style.context(('ggplot')):
    plt.plot(wl, X2.T)
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('D2 Absorbance')
plt.show()

### 2.2.4 Function: PLSR optimization

In [ ]:
# source: https://nirpyresearch.com/partial-least-squares-regression-python/
def optimise_pls_cv(X, y, n_comp, plot_components=True):
    '''Run PLS including a variable number of components, up to n_comp,
       and calculate MSE '''
    mse = []
    component = np.arange(1, n_comp)
    for i in component:
        pls = PLSRegression(n_components=i)
        # Cross-validation
        y_cv = cross_val_predict(pls, X, y, cv=10)
        mse.append(mean_squared_error(y, y_cv))
        comp = 100*(i+1)/40
        # Trick to update status on the same line
        stdout.write("\r%d%% completed" % comp)
        stdout.flush()
    stdout.write("\n")
    # Calculate and print the position of minimum in MSE
    msemin = np.argmin(mse)
    print("Suggested number of components: ", msemin+1)
    stdout.write("\n")
    if plot_components is True:
        with plt.style.context(('ggplot')):
            plt.plot(component, np.array(mse), '-v', color = 'blue', mfc='blue')
            plt.plot(component[msemin], np.array(mse)[msemin], 'P', ms=10, mfc='red')
            plt.xlabel('Number of PLS components')
            plt.ylabel('MSE')
            plt.title('PLS')
            plt.xlim(left=-1)
        plt.show()
    # Define PLS object with optimal number of components
    pls_opt = PLSRegression(n_components=msemin+1)
    # Fir to the entire dataset
    pls_opt.fit(X, y)
    y_c = pls_opt.predict(X)
    # Cross-validation
    y_cv = cross_val_predict(pls_opt, X, y, cv=10)
    # Calculate scores for calibration and cross-validation
    score_c = r2_score(y, y_c)
    score_cv = r2_score(y, y_cv)
    # Calculate mean squared error for calibration and cross validation
    mse_c = mean_squared_error(y, y_c)
    mse_cv = mean_squared_error(y, y_cv)
    print('R2 calib: %5.3f'  % score_c)
    print('R2 CV: %5.3f'  % score_cv)
    print('MSE calib: %5.3f' % mse_c)
    print('MSE CV: %5.3f' % mse_cv)
    # Plot regression and figures of merit
    rangey = max(y) - min(y)
    rangex = max(y_c) - min(y_c)
    # Fit a line to the CV vs response
    z = np.polyfit(y, y_c, 1)
    with plt.style.context(('ggplot')):
        fig, ax = plt.subplots(figsize=(9, 5))
        ax.scatter(y_c, y, c='red', edgecolors='k')
        #Plot the best fit line
        ax.plot(np.polyval(z,y), y, c='blue', linewidth=1)
        #Plot the ideal 1:1 line
        ax.plot(y, y, color='green', linewidth=1)
        plt.title('$R^{2}$ (CV): '+str(score_cv))
        plt.xlabel('Predicted $^{\circ}$Brix')
        plt.ylabel('Measured $^{\circ}$Brix')
        plt.show()
return

### 2.2.5 Fitt the best PLSR

In [ ]:
optimise_pls_cv(X,Y, 40, plot_components=True)

pls = PLSRegression(n_components=7)
pls = pls.fit(X, Y)

### 2.2.6 Fuction: VIP calcuation

In [ ]:
# VIP calculation
# sources :(Used)https://www.researchgate.net/post/How_can_I_compute_Variable_Importance_in_Projection_VIP_in_Partial_Least_Squares_PLS
# (possible) https://github.com/scikit-learn/scikit-learn/issues/7050

def _calculate_vips(model):
    t = model.x_scores_
    w = model.x_weights_
    q = model.y_loadings_
    p, h = w.shape
    vips = np.zeros((p,))
    s = np.diag(np.matmul(np.matmul(np.matmul(t.T,t),q.T), q)).reshape(h, -1)
    total_s = np.sum(s)
    for i in range(p):
        weight = np.array([ (w[i,j] / np.linalg.norm(w[:,j]))**2 for j in range(h) ])
        vips[i] = np.sqrt(p*(np.matmul(s.T, weight))/total_s)
    return vips

In [ ]:
# vip calculationa and plotting

vip = _calculate_vips(pls)
plt.figure(figsize=(8,4.5))
with plt.style.context(('ggplot')):
    plt.plot(wl, vip.T)
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('VIP')
plt.show()

## 2.3 Ridge regression 

In [ ]:
 # Define the parameters and their range
parameters = {'alpha':np.logspace(-4, -3.5, 50)}

In [ ]:
# Run a Grid search, using R^2 as the metric to optimize alpha
ridge = GridSearchCV(linear_model.Ridge(), parameters, scoring ='r2', cv = 10)

#fit to the data
ridge.fit(X, Y)

#Get the optimised value of alpha
print('Best parameter alpha = ', ridge.best_params_['alpha'])
print('R2 calibration: %5.3f'  % ridge.score(X,Y))
# Run a ridge regression with the optimised value
ridge1 = linear_model.Ridge(alpha=ridge.best_params_['alpha'])
y_cv = cross_val_predict(ridge1, X, Y, cv=10)
# y_cv=predicted
score_cv = r2_score(Y, y_cv)
mse_cv = mean_squared_error(Y, y_cv)
print('R2 CV (Ridge): %5.3f'  % score_cv)
print('MSE CV (Ridge): %5.3f' % mse_cv)

In [ ]:
def rfr_model(X, y):
# Perform Grid-Search
    gsc = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid={
            'max_depth': range(3,7),
            'n_estimators': (10, 50, 100, 1000),
        },
        cv=5, scoring='neg_mean_squared_error', verbose=0,                         n_jobs=-1)
    
    grid_result = gsc.fit(X, y)
    best_params = grid_result.best_params_
    
    rfr = RandomForestRegressor(max_depth=best_params["max_depth"], n_estimators=best_params["n_estimators"],                               random_state=False, verbose=False)
# Perform K-Fold CV
    scores = cross_val_score(rfr, X, y, cv=10, scoring='neg_mean_absolute_error')

    return scores

In [ ]:
#rfr = rfr_model(X, Y)
score = cross_val_score(rfr, X, Y, cv=10, scoring='neg_mean_absolute_error')

In [ ]:
predicitons = cross_val_predict(rfr, X, Y, cv=10)

## 2.4 RF

### 2.4.1 Select parameter and fit the best RF

In [ ]:

# Create the parameter grid based on the results of random search 
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 42)

param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

In [ ]:
# fit RF and pring the best parameters
grid_search.fit(X_train, Y_train)
grid_search.best_params_

### 2.4.2 Function: Evaluate RF

In [ ]:

def evaluate(model, X_test, Y_test):
    predictions = model.predict(X_test)
    errors = abs(predictions - Y_test)
    mape = 100 * np.mean(errors / Y_test)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [ ]:
best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_test, Y_test)

## 2.5 SVM

### 2.5.1 Select paramters and fit the best SVM

In [ ]:
np.random.seed(0)

parameters = {'kernel': ('linear', 'rbf','poly'), 'C':[1.5, 10],'gamma': [1e-7, 1e-4],'epsilon':[0.1,0.2,0.5,0.3]}
svr = SVR()
grid_search = GridSearchCV(svr, parameters, cv=5)
grid_search.fit(X_train,Y_train)
grid_search.best_params_

best_grid = grid_search.best_estimator_
grid_accuracy = evaluate(best_grid, X_test, Y_test)

In [ ]:
from sklearn_extensions.extre_leraning_mashines.elm import ELMRegressor

## 2.6 ELR

### 2.6.1 Fuctions: Activation and Main ELR

In [ ]:

def act_fun(x, actf):

    # Calculate hidden neuron output matrix H
    # select activation initially
    if actf == 'tribas':
        # triangular activation function
        x = x.astype(float)
        H = np.clip(1.0 - np.fabs(x), 0.0, 1.0)
    elif actf == 'sig':
        # sigmoid activation function
        x = x.astype(float)
        H = 1.0/(1.0 + np.exp(-x))
    elif actf == 'hard_limit':
        # hard limit actf function
        x = x.astype(float)
        H = np.array(x > 0.0, dtype=float)
    elif actf == 'Gaussian':
        # gaussian RBF
        x = x.astype(float)
        H = np.exp(-pow(x, 2.0))
    elif actf == 'dual':
        x = x.astype(float)
        H = np.array(x > 0.0, dtype=float)*x + np.array(x <= 0.0, dtype=float)*np.tan(x)
    return H

def ELR_main(trainX, testX, trainY, testY, actf, NumberofHiddenNeurons):

    NumberofInputNeurons = trainX.shape[1]

    trainX = np.transpose(trainX)
    trainY = np.transpose(trainY)
    testX  = np.transpose(testX)
    testY  = np.transpose(testY)
    
    # Random generate input weights InputWeight (w_i) and biases BiasofHiddenNeurons (b_i) of hidden neurons
    np.random.seed(0)
    InputWeight = np.random.random((NumberofHiddenNeurons,NumberofInputNeurons))*2-1
    BiasofHiddenNeurons = np.random.random((NumberofHiddenNeurons,1))
    tempH = np.dot(InputWeight, trainX)
    # add bias 
    tempH = tempH + BiasofHiddenNeurons
    H = act_fun(tempH, actf)
    # Calculate output weights OutputWeight
    OutputWeight = np.dot(np.linalg.pinv(np.transpose(H)), np.transpose(trainY))  # option 1
                    
    #Calculate the training pred
    # Y_pred_train = np.transpose(np.dot(np.transpose(H), OutputWeight))
    
    #Calculate the output of testing input
    tempH_test = np.dot(InputWeight, testX)
    tempH_test = tempH_test + BiasofHiddenNeurons
    H_test = act_fun(tempH_test, actf)
    
    # TY: the actual output of the testing data
    y_pred = np.transpose(np.dot(np.transpose(H_test), OutputWeight))
    return y_pred


### 2.6.2 Load and parse data, and fit ELR

In [ ]:

df = pd.read_csv('test.csv')
dataset = df.values
X = dataset[:, 1:601]
Y = dataset[:, 602]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

#method = 'ELR'
actf = 'hard_limit'

R2_list = []

for NumberofHiddenNeurons in range (1, 201):
    y_pred = ELR_main(X_train, X_test, Y_train, Y_test, actf, NumberofHiddenNeurons)
    R2 = r2_score(Y_test,y_pred)
    R2_list.append(R2)
    
print ("The list of R2 is:", R2_list)

maximum = R2_list[0]
for i in range (0, 200):
    if R2_list[i] > maximum:
        maximum = R2_list[i]
        bestNumberofNeurons = i+1
print ("The best number of neuron for the prediction is:", bestNumberofNeurons)
        
theBestR2 = maximum
print ("The best R2 is:", theBestR2)

#NumberofHidddenNeurons = bestNumberofNeurons 
   
y_pred = ELR_main(X_train, X_test, Y_train, Y_test, actf, bestNumberofNeurons)

MSE = mean_squared_error(Y_test,y_pred)
print("Mean Squared Error: %.2f" % MSE)

R2 = r2_score(Y_test,y_pred)
print("R2:%.2f" %R2)

#MSE = mean_squared_error(testY,y_pred)
#print("Mean Squared Error: %.2f" % MSE)

#R2 = r2_score(testY,y_pred)
#print("R2:%.2f" %R2)

## Plot outputs


### 2.6.3 Visualize ELR the best resutls

In [ ]:
from matplotlib.pyplot import text

MSE = mean_squared_error(Y_test,y_pred)
print("Mean Squared Error: %.2f" % MSE)

R2 = r2_score(Y_test,y_pred)
print("R2:%.2f" %R2)

fig, ax = plt.subplots(figsize = (8, 6))
ax.scatter(Y_test, y_pred, edgecolors=(0, 0, 0)) # testY is ground truth for testing data, y_pred is the predicted output
ax.plot([Y_test.min(), Y_test.max()], [Y_test.min(), Y_test.max()], 'b--', lw=2)
#text(int(Y_test.min())+1, int(Y_test.min())+24, 'R2=' + str(round(R2,2)), bbox=dict(facecolor='red', alpha=1))
# print MSE on the figure
#text(int(Y_test.min())+1, int(Y_test.min())+21, 'MSE=' + str(round(MSE,2)), bbox=dict(facecolor='red', alpha=0))
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

